In [8]:
using Distributions

## Parameters to get S
Get the parameters for the N that is our frequency random varible and we know it distributes as N $\sim$ NegativeBinomial(r, p) but we also know that $\mathbb{E} = 140$ and also that $\mathbb{V} = 15²$.

Doing a little reasherch we can get that the Expected value of N is $\frac{r(1-p)}{p}$ and its variance is $\mathbb{V}(N) = \frac{r(1-p)}{p²}$
So it's only left to plug-in our values and solve for $r$ and $p$. That gives us that $r = 3920/17$ and $p = 28/45$ and we don't need to round the value of r beacuse in julia it's an expression for the distribution in case $r \in \mathbb{R}$

In [2]:
mu = 3.912115; sigma = 2.71873; 

freq = NegativeBinomial(3920/17 ,28/45)
sev = LogNormal(mu, sigma)

LogNormal{Float64}(μ=3.912115, σ=2.71873)

In [4]:
function  simul_events(n)
    S = zeros(n)
    N = rand(freq, n)
    
    N_pos = findall(N .> 0)
    
    for i in N_pos
        Yi = rand(sev, N[i])
        S[i] = sum(Yi)
    end
    return(S, N)
end

simul_events (generic function with 1 method)

In [ ]:
S = simul_events(10_000)

The next step is to fit the 